In [ ]:
!pip install -q scikit-learn xgboost pyarrow

import os, numpy as np, pandas as pd
from google.colab import drive

BASE_DIR   = "/content/drive/MyDrive/Projects/JPM Short Interest/Wrapup/outputs"
DATA_FILES = ["merged_with_nolbert.parquet",
              "merged_with_nolbert_new.parquet"]  # <- both files here

drive.mount('/content/drive')

def load_any(path):
    return pd.read_parquet(path) if path.endswith(".parquet") else pd.read_csv(path, low_memory=False)

# load all, align columns, then union
paths = [os.path.join(BASE_DIR, f) for f in DATA_FILES]
dfs   = [load_any(p) for p in paths]

# union columns
all_cols = sorted(set().union(*(d.columns for d in dfs)))
dfs = [d.reindex(columns=all_cols) for d in dfs]
df  = pd.concat(dfs, ignore_index=True)

# choose key columns for dedup (prefer symbolCode, else ticker)
key_candidates = [["symbolCode","interval"], ["ticker","interval"], ["interval"]]
key_cols = next((ks for ks in key_candidates if set(ks).issubset(df.columns)), None)

# prefer the most recent row per key based on settlementDate / event_dt if present
if key_cols:
    # build a recency sort key
    sk = []
    if "settlementDate" in df.columns:
        df["settlementDate"] = pd.to_datetime(df["settlementDate"], errors="coerce")
        sk.append(df["settlementDate"])
    if "event_dt" in df.columns:
        df["event_dt"] = pd.to_datetime(df["event_dt"], errors="coerce")
        sk.append(df["event_dt"])
    if sk:
        df = (df.assign(_sortkey=pd.concat(sk, axis=1).max(axis=1))
                .sort_values(key_cols + ["_sortkey"])
                .drop_duplicates(subset=key_cols, keep="last")
                .drop(columns=["_sortkey"]))
    else:
        df = df.drop_duplicates(subset=key_cols, keep="last")

print("Loaded union shape:", df.shape)
print("Sample cols:", df.columns.tolist()[:25])
assert "interval" in df.columns, "interval missing"
if "currentShortPositionQuantity" not in df.columns:
    raise ValueError("currentShortPositionQuantity missing (needed for target/baselines)")


Mounted at /content/drive
Loaded union shape: (188, 25)
Sample cols: ['NEGATIVE', 'NEUTRAL', 'POSITIVE', 'accountingYearMonthNumber', 'averageDailyVolumeQuantity', 'changePercent', 'changePreviousNumber', 'currentShortPositionQuantity', 'daysToCoverQuantity', 'doc_id', 'event_dt', 'full_text', 'interval', 'issueName', 'issuerServicesGroupExchangeCode', 'marketClassCode', 'previousShortPositionQuantity', 'revisionFlag', 'sent_entropy', 'sent_polarity', 'settlementDate', 'stockSplitFlag', 'target_next', 'ticker', 'transcriptid']


In [ ]:
import os, pandas as pd, numpy as np, re
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR  = "/content/drive/MyDrive/Projects/JPM Short Interest/Wrapup/outputs"  # <- adjust if needed
DATA_FILES = ["merged_with_nolbert.parquet",
              "merged_with_nolbert_new.parquet"]

print("Folder listing:")
print(os.listdir(BASE_DIR))

path = os.path.join(BASE_DIR, DATA_FILE)
df = pd.read_parquet(path) if path.endswith(".parquet") else pd.read_csv(path, low_memory=False)
print("Loaded:", path, "shape:", df.shape)
print("Columns:", df.columns.tolist())
print(df.head(3))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder listing:
['merged_with_nolbert.parquet', 'merged_docs_new.parquet', 'merged_chunks_nltk_new.parquet', 'merged_with_nolbert_new.parquet']
Loaded: /content/drive/MyDrive/Projects/JPM Short Interest/Wrapup/outputs/merged_with_nolbert.parquet shape: (108, 25)
Columns: ['ticker', 'interval', 'event_dt', 'transcriptid', 'full_text', 'accountingYearMonthNumber', 'issueName', 'issuerServicesGroupExchangeCode', 'marketClassCode', 'currentShortPositionQuantity', 'previousShortPositionQuantity', 'stockSplitFlag', 'averageDailyVolumeQuantity', 'daysToCoverQuantity', 'revisionFlag', 'changePercent', 'changePreviousNumber', 'settlementDate', 'doc_id', 'NEUTRAL', 'POSITIVE', 'NEGATIVE', 'sent_polarity', 'sent_entropy', 'target_next']
  ticker  interval            event_dt  transcriptid  \
0   PLTR         0 2023-08-10 16:33:37     2878274.0   
1   QBTS         0 2023

In [ ]:
# ---- Coerce dtypes: run RIGHT AFTER loading df ----
import re

def to_num(s):
    if pd.isna(s): return np.nan
    if isinstance(s,(int,float,np.number)): return float(s)
    s = str(s).strip()
    s = re.sub(r'[,%$()]', '', s)
    try: return float(s)
    except: return np.nan

num_cols = [
    "currentShortPositionQuantity","previousShortPositionQuantity",
    "averageDailyVolumeQuantity","daysToCoverQuantity",
    "changePercent","changePreviousNumber","si_ratio_to_adv",
    "NEUTRAL","POSITIVE","NEGATIVE","sent_polarity","sent_entropy",
    "si_pct_change"  # if already present
]
num_cols = [c for c in num_cols if c in df.columns]

for c in num_cols:
    df[c] = df[c].map(to_num).astype(float)

for c in ["settlementDate","event_dt"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

if "interval" in df.columns:
    df["interval"] = pd.to_numeric(df["interval"], errors="coerce").astype("Int64")

print(df[num_cols].describe().T)
print("Null ratios (selected):")
print(df[["interval"] + num_cols].isna().mean().sort_values(ascending=False).head(20))

                               count          mean           std  \
currentShortPositionQuantity   108.0  3.774650e+07  4.131273e+07   
previousShortPositionQuantity  108.0  3.730857e+07  4.101285e+07   
averageDailyVolumeQuantity     108.0  3.492653e+07  3.666962e+07   
daysToCoverQuantity            108.0  2.026944e+00  1.465278e+00   
changePercent                  108.0  5.890093e+00  2.373872e+01   
changePreviousNumber           108.0  4.379245e+05  6.271043e+06   
NEUTRAL                        108.0  5.663359e-01  1.322819e-01   
POSITIVE                       108.0  3.654293e-01  1.260092e-01   
NEGATIVE                       108.0  6.823475e-02  4.225318e-02   
sent_polarity                  108.0  2.971946e-01  1.335245e-01   
sent_entropy                   108.0  8.233165e-01  1.101123e-01   

                                        min           25%           50%  \
currentShortPositionQuantity   1.603041e+06  3.746695e+06  2.049850e+07   
previousShortPositionQuantity  1.

In [ ]:
def add_lags(g, cols, lags=(1,2,3)):
    for c in cols:
        for L in lags:
            g[f"{c}_lag{L}"] = g[c].shift(L)
    return g

def add_rolls(g, cols, win=3):
    for c in cols:
        g[f"{c}_rollmean{win}"] = g[c].rolling(win).mean()
        g[f"{c}_rollstd{win}"]  = g[c].rolling(win).std()
    return g

def _fe_symbol(g, base_cols, interval_col="interval"):
    g = g.sort_values(interval_col)
    g = add_lags(g, base_cols, lags=(1,2,3))
    g = add_rolls(g, ["currentShortPositionQuantity","averageDailyVolumeQuantity","daysToCoverQuantity"], win=3)
    # lag sentiment once to test predictive power
    if "sent_polarity" in g.columns:
        g["sent_polarity_lag1"] = g["sent_polarity"].shift(1)
    return g

def build_features(df, symbol_col="symbolCode", interval_col="interval"):
    # base fields expected; create si_pct_change if missing
    if "si_pct_change" not in df.columns:
        df["si_pct_change"] = (
            df["currentShortPositionQuantity"] - df["previousShortPositionQuantity"]
        ) / df["previousShortPositionQuantity"].replace(0, np.nan)

    base_cols = [
        "currentShortPositionQuantity","averageDailyVolumeQuantity","daysToCoverQuantity",
        "si_ratio_to_adv","si_pct_change","sent_polarity","sent_entropy",
        "POSITIVE","NEGATIVE","NEUTRAL"
    ]
    for c in base_cols:
        if c not in df.columns:
            df[c] = np.nan

    if symbol_col in df.columns:
        df = df.groupby(symbol_col, group_keys=False).apply(lambda g: _fe_symbol(g, base_cols, interval_col))
    else:
        df = _fe_symbol(df, base_cols, interval_col)

    # simple interactions
    df["polxratio"] = df["sent_polarity"] * df["si_ratio_to_adv"]
    df["negxdtc"]   = df["NEGATIVE"] * df["daysToCoverQuantity"]
    return df

# build features
df = build_features(df)

# target: next interval SI level (create if missing)
if "target_next" not in df.columns:
    if "symbolCode" in df.columns:
        df["target_next"] = df.groupby("symbolCode")["currentShortPositionQuantity"].shift(-1)
    else:
        df["target_next"] = df["currentShortPositionQuantity"].shift(-1)

# drop rows with NA from lags/target
model_df = df.dropna().sort_values(["symbolCode","interval"] if "symbolCode" in df.columns else ["interval"]).reset_index(drop=True)

print("model_df shape:", model_df.shape)

model_df shape: (0, 65)


In [ ]:
# Build si_pct_change if missing
if "si_pct_change" not in df.columns and \
   {"currentShortPositionQuantity","previousShortPositionQuantity"}.issubset(df.columns):
    prev = df["previousShortPositionQuantity"].replace(0, np.nan)
    df["si_pct_change"] = (df["currentShortPositionQuantity"] - df["previousShortPositionQuantity"]) / prev

# Target: next short interest level
if "target_next" not in df.columns:
    if "symbolCode" in df.columns:
        df["target_next"] = df.groupby("symbolCode")["currentShortPositionQuantity"].shift(-1)
    else:
        df["target_next"] = df["currentShortPositionQuantity"].shift(-1)

# Keep only rows necessary for modeling (ESSENTIALS ONLY)
must_have = ["target_next","currentShortPositionQuantity"]
for c in ["averageDailyVolumeQuantity","daysToCoverQuantity","sent_polarity","NEGATIVE","POSITIVE"]:
    if c in df.columns: must_have.append(c)

model_df = (df.dropna(subset=must_have)
              .sort_values(["symbolCode","interval"] if "symbolCode" in df.columns else ["interval"])
              .reset_index(drop=True))

print("Rows after minimal dropna:", len(model_df))
print("Intervals span:", model_df["interval"].min() if "interval" in model_df else None,
      "→", model_df["interval"].max() if "interval" in model_df else None)


Rows after minimal dropna: 108
Intervals span: 0 → 52


In [ ]:
keep = [c for c in ["interval","symbolCode","settlementDate","currentShortPositionQuantity",
                    "previousShortPositionQuantity","averageDailyVolumeQuantity","daysToCoverQuantity",
                    "NEUTRAL","POSITIVE","NEGATIVE","sent_polarity","sent_entropy","target_next"] if c in df.columns]
print(df[keep].head(10))
print(df[keep].tail(10))

   interval settlementDate  currentShortPositionQuantity  \
0         0     2023-08-15                   135778479.0   
1         0     2023-08-15                     2344710.0   
2         0     2023-08-15                     4350805.0   
3         2     2023-08-31                     1773374.0   
4         2     2023-08-31                     2679864.0   
5         2     2023-08-31                   155590455.0   
6         3     2023-09-15                   154030736.0   
7         3     2023-09-15                     2323307.0   
8         3     2023-09-15                     2084243.0   
9         4     2023-09-29                     1603041.0   

   previousShortPositionQuantity  averageDailyVolumeQuantity  \
0                    119438357.0                  86382480.0   
1                      2681290.0                  10651495.0   
2                      2179662.0                   4319256.0   
3                      2344710.0                   3491994.0   
4                  

In [ ]:
def add_lags(g, cols, lags=(1,)):
    for c in cols:
        for L in lags:
            g[f"{c}_lag{L}"] = g[c].shift(L)
    return g

def add_rolls(g, cols, win=2):
    for c in cols:
        g[f"{c}_rollmean{win}"] = g[c].rolling(win).mean()
        g[f"{c}_rollstd{win}"]  = g[c].rolling(win).std()
    return g

base_cols = [c for c in [
    "currentShortPositionQuantity","averageDailyVolumeQuantity","daysToCoverQuantity",
    "si_ratio_to_adv","si_pct_change","sent_polarity","sent_entropy","POSITIVE","NEGATIVE","NEUTRAL"
] if c in model_df.columns]

def _fe_symbol(g):
    g = g.sort_values("interval" if "interval" in g.columns else g.index)
    g = add_lags(g, base_cols, lags=(1,))
    g = add_rolls(g, [c for c in ["currentShortPositionQuantity","averageDailyVolumeQuantity","daysToCoverQuantity"] if c in g.columns], win=2)
    if "sent_polarity" in g.columns:
        g["sent_polarity_lag1"] = g["sent_polarity"].shift(1)
    return g

if "symbolCode" in model_df.columns:
    model_df = model_df.groupby("symbolCode", group_keys=False).apply(_fe_symbol)
else:
    model_df = _fe_symbol(model_df)

# Drop rows that became NA due to lags/rolls (only now)
model_df = model_df.dropna(subset=["target_next","currentShortPositionQuantity"]).reset_index(drop=True)
print("Rows after light FE:", len(model_df))

Rows after light FE: 108


In [ ]:
exclude = {"target_next","full_text","settlementDate","event_dt","interval","transcriptid","doc_id"}
if "symbolCode" in model_df.columns:
    exclude.add("symbolCode")

candidates = [c for c in model_df.columns if c not in exclude]
X = model_df[candidates].select_dtypes(include=["number"]).copy()

# Fallback core features if nothing left:
if X.shape[1] == 0:
    core = [c for c in ["currentShortPositionQuantity","averageDailyVolumeQuantity","daysToCoverQuantity",
                        "si_ratio_to_adv","si_pct_change","sent_polarity","NEGATIVE","POSITIVE"] if c in model_df.columns]
    X = model_df[core].astype(float)

y = model_df["target_next"].astype(float)
y_curr = model_df["currentShortPositionQuantity"].astype(float).to_numpy()

# Clean X: drop all-NA columns, replace infs with NaN (will be imputed)
X = X.replace([np.inf, -np.inf], np.nan)
X = X.loc[:, X.notna().any(axis=0)]  # drop columns entirely NaN
print("After cleaning, X shape:", X.shape)

print("Final X shape:", X.shape, "y len:", len(y))

After cleaning, X shape: (108, 52)
Final X shape: (108, 52) y len: 108


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
n_samples = len(X)
if n_samples < 3:
    raise ValueError(f"Not enough samples ({n_samples}). Expand date range or reduce FE.")
n_splits = max(2, min(5, n_samples - 1))
tscv = TimeSeriesSplit(n_splits=n_splits)
print("Using n_splits =", n_splits)

Using n_splits = 5


In [ ]:
# === Add BELOW your "Using n_splits = ..." print ===
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
import time
from sklearn.impute import SimpleImputer

def metrics(y_true, y_pred, y_curr):
    # RMSE compatible with older sklearn (no 'squared' kwarg)
    # directional accuracy on change
    dy_true = np.sign(y_true - y_curr)
    dy_pred = np.sign(y_pred - y_curr)
    dir_acc = float((dy_true == dy_pred).mean())
    return {"DirAcc": dir_acc}


# Baseline: persistence (predict current SI level)
baseline = metrics(y.to_numpy(), y_curr, y_curr)
print("Baseline (persistence):", baseline)

# Models
models = [
    ("ElasticNet", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler",  StandardScaler(with_mean=False)),
        ("model",   ElasticNet(alpha=1e-3, l1_ratio=0.3, max_iter=3000, random_state=42))
    ])),
    ("XGBoost", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),  # XGB can handle NaN, but median impute is fine too
        ("model",   XGBRegressor(
            n_estimators=800, max_depth=5, subsample=0.8, colsample_bytree=0.8,
            learning_rate=0.03, reg_lambda=1.0, objective="reg:squarederror", random_state=42
        ))
    ]))
]

# Time-based CV
cv_rows = []
splits = list(tscv.split(X))
for name, mdl in models:
    for fold, (tr, te) in enumerate(splits):
        X_tr, X_te = X.iloc[tr], X.iloc[te]
        y_tr, y_te = y.iloc[tr], y.iloc[te]
        y_curr_te  = y_curr[te]
        mdl.fit(X_tr, y_tr)
        y_hat = mdl.predict(X_te)
        sc = metrics(y_te.to_numpy(), y_hat, y_curr_te)
        sc["model"], sc["fold"] = name, fold
        cv_rows.append(sc)

cv = pd.DataFrame(cv_rows)
print("\nCV means by model:\n", cv.groupby("model")[["DirAcc"]].mean())

# Holdout = last split, use XGB if present (else ElasticNet)
best_name, best_model = models[-1]  # XGBoost
last_tr, last_te = splits[-1]
best_model.fit(X.iloc[last_tr], y.iloc[last_tr])
y_hat = best_model.predict(X.iloc[last_te])
hold = metrics(y.iloc[last_te].to_numpy(), y_hat, y_curr[last_te])
print("\nHoldout:", hold)

# Optional: feature importance (XGBoost)
if hasattr(best_model, "feature_importances_"):
    imp = pd.Series(best_model.feature_importances_, index=X.columns).sort_values(ascending=False)
    print("\nTop 20 features:\n", imp.head(20))

# Optional: save artifacts next to your data file
ts = time.strftime("%Y%m%d_%H%M%S")
OUT_DIR = os.path.join(BASE_DIR, "models_nb", ts)
os.makedirs(OUT_DIR, exist_ok=True)

cv.to_csv(os.path.join(OUT_DIR, "cv_metrics.csv"), index=False)
pd.DataFrame([baseline]).to_csv(os.path.join(OUT_DIR, "baseline_metrics.csv"), index=False)
pd.DataFrame([hold]).to_csv(os.path.join(OUT_DIR, "holdout_metrics.csv"), index=False)

preds = model_df.iloc[last_te][["interval"] + (["symbolCode"] if "symbolCode" in model_df.columns else [])].copy()
preds["y_true"] = y.iloc[last_te].to_numpy()
preds["y_pred"] = y_hat
preds.to_parquet(os.path.join(OUT_DIR, "holdout_predictions.parquet"), index=False)

if 'imp' in locals():
    imp.to_csv(os.path.join(OUT_DIR, "feature_importance.csv"), header=["importance"])

print("\nArtifacts saved to:", OUT_DIR)

Baseline (persistence): {'DirAcc': 0.0}


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.392e+13, tolerance: 8.527e+12
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.193e+15, tolerance: 1.182e+13
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.435e+16, tolerance: 1.365e


CV means by model:
               DirAcc
model               
ElasticNet  0.577778
XGBoost     0.777778

Holdout: {'DirAcc': 0.8333333333333334}

Artifacts saved to: /content/drive/MyDrive/Projects/JPM Short Interest/Wrapup/outputs/models_nb/20251009_192534
